**ЗАДАНИЕ 1**

Для датасета с 18000 новостей, сгруппированных по 20 темам, выберете 4 темы и используйте модель RNN для классификации новостей (например, "спорт", "политика", "технологии" и т.д.). Объясните, как работает RNN модель в задачах классификации текстов, какие предобработки данных вы применяете и какие результаты вы получили в процессе обучения и тестирования модели

**RNN (Recurrent Neural Network)** - это тип нейронной сети, способной обрабатывать последовательные данные, такие как текст.

В задачах классификации текстов RNN может быть использована для анализа и учета последовательной структуры текста. Вот как работает RNN модель в задачах классификации текстов:

1. Последовательный вход: Текст представляется в виде последовательности слов или токенов. Каждое слово кодируется в числовой формат, обычно с использованием векторных представлений, таких как word embeddings (например, Word2Vec или GloVe).
2. Рекуррентные слои: RNN содержит один или несколько рекуррентных слоев. Эти слои позволяют модели учитывать контекст и зависимости между словами в тексте. Каждый слой принимает на вход текущее слово и состояние, полученное из предыдущего слова, и выдает новое состояние.
3. Предсказание класса: Последний состояний рекуррентного слоя или выход последнего временного шага передается через полносвязанный (Dense) слой с функцией активации, которая соответствует числу классов в задаче классификации. Это выходной слой, который предсказывает класс, к которому принадлежит текст.
4. Обучение: Модель обучается с использованием обучающих данных и функции потерь, такой как кросс-энтропия. Обратное распространение ошибки используется для настройки весов модели таким образом, чтобы минимизировать потери.

Преимущества RNN в задачах классификации текстов:

 • Способность учитывать контекст и зависимости между словами, что позволяет лучше анализировать смысл текста.
 • Подходит для текстов разной длины, так как RNN обрабатывает последовательности переменной длины.

Однако у RNN есть и недостатки, такие как проблема исчезающего градиента. Поэтому в практике часто используются усовершенствованные архитектуры, такие как LSTM (Long Short-Term Memory) и GRU (Gated Recurrent Unit), чтобы бороться с этими ограничениями и улучшить производительность в задачах классификации текстов.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc',
              'comp.graphics', 'sci.space']

newsgroups = fetch_20newsgroups(subset='all',
                                      categories=categories, remove=('headers', 'footers', 'quotes'))
newsgroups.filenames.shape

(3387,)

In [ ]:
print(newsgroups.data[0])

My point is that you set up your views as the only way to believe.  Saying 
that all eveil in this world is caused by atheism is ridiculous and 
counterproductive to dialogue in this newsgroups.  I see in your posts a 
spirit of condemnation of the atheists in this newsgroup bacause they don'
t believe exactly as you do.  If you're here to try to convert the atheists 
here, you're failing miserably.  Who wants to be in position of constantly 
defending themselves agaist insulting attacks, like you seem to like to do?!
I'm sorry you're so blind that you didn't get the messgae in the quote, 
everyone else has seemed to.


In [ ]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Выбор только необходимых тем
filtered_data = []
filtered_labels = []
for i, topic in enumerate(categories):
    mask = np.where(newsgroups.target == i)
    filtered_data.extend(np.array(newsgroups.data)[mask])
    filtered_labels.extend([i] * len(mask[0]))

# Предобработка данных
max_words = 10000  # Максимальное количество слов в словаре
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(filtered_data)
X = tokenizer.texts_to_sequences(filtered_data)
X = pad_sequences(X)

# Преобразование меток в one-hot кодировку
y = np.array(filtered_labels)
num_classes = len(categories)
y = tf.keras.utils.to_categorical(y, num_classes)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=X.shape[1]))
model.add(SimpleRNN(128))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/5
43/43 [==============================] - 651s 15s/step - loss: 1.3805 - accuracy: 0.2997 - val_loss: 1.3704 - val_accuracy: 0.3127
Epoch 2/5
43/43 [==============================] - 648s 15s/step - loss: 1.3383 - accuracy: 0.3606 - val_loss: 1.3487 - val_accuracy: 0.3142
Epoch 3/5
43/43 [==============================] - 640s 15s/step - loss: 1.3346 - accuracy: 0.3614 - val_loss: 1.3545 - val_accuracy: 0.2920
Epoch 4/5
43/43 [==============================] - 645s 15s/step - loss: 1.3149 - accuracy: 0.3957 - val_loss: 1.3482 - val_accuracy: 0.3776
Epoch 5/5
43/43 [==============================] - 645s 15s/step - loss: 1.2830 - accuracy: 0.4710 - val_loss: 1.3176 - val_accuracy: 0.3761


Точность проверки на валидационных данных показывает, насколько хорошо модель способна обобщать свои предсказания на неизвестные данные. Так как оценка эффективности довольно таки низкая, то мы можем сказать, что модель плохо обощает свои предскзания на неизвестные данные.

На 3 эпохе в связи с понижением валидационной оценки модели, происходит недообучение, так же об этом говорит ошибка, которая выросла. На 4 эпохе все нормализуется, ошибка падает, оценка растет.

Общая accuracy должна быть выше валидационного, что у нас и имеется, тем не менее она тоже имеет низкое значение, это значит, что наша модель плохо обучена (недообучена)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

22/22 [==============================] - 16s 747ms/step - loss: 1.3176 - accuracy: 0.3761
Loss: 1.3175781965255737, Accuracy: 0.3761062026023865


**ЗАДАНИЕ 2**

используя алгоритмы, такие как Word2Vec или GloVe, необходимо построить эмбеддинги для вашего датасета (по заданию 1). Вывести примеры «близких» слов. Сделайте выводы

In [ ]:
pip install gensim

In [ ]:
from gensim.models import Word2Vec

tokenized_data = [text.split() for text in filtered_data]

word2vec_model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, sg=0)

In [ ]:
sports_vector = word2vec_model.wv['sports']

similar_words = word2vec_model.wv.most_similar('sports', topn=5)

for word, score in similar_words:
    print(f"Слово: {word}, Близость: {score}")

Слово: Revelation, Близость: 0.9721689820289612
Слово: Arianespace, Близость: 0.9684611558914185
Слово: VIKING, Близость: 0.9683914184570312
Слово: Cassini, Близость: 0.967854380607605
Слово: orbiter, Близость: 0.9665995836257935


По резульататам предыдущего кода мы можем увидеть, что качетсво наших эмбеддингов очень хорошее, так как близость высокая.